In [ ]:
!git clone https://github.com/lilnyan/detection-demo
!mkdir templates
!mkdir static
shutil.move("/content/detection-demo/templates/index.jinja2", "/content/templates/")
shutil.move("/content/detection-demo/static/favicon.ico", "/content/static/")
shutil.move("/content/detection-demo/static/index.js", "/content/static/")
shutil.move("/content/detection-demo/index.html", "/content/")
shutil.move("/content/detection-demo/requirements.txt", "/content/")
!pip install -r requirements.txt

In [ ]:
#Установка необходимых пакетов
!pip install flask-ngrok
!pip install jsonpickle

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, Response, render_template, jsonify
import jsonpickle
from PIL import Image
import io, sys
import numpy as np
import cv2
import base64

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def hello_world():
    return render_template('index.jinja2')

@app.route('/maskImage' , methods=['POST'])
def mask_image():
    # print(request.files , file=sys.stderr)
    file = request.files['image'].read() ## byte file

    npimg = np.fromstring(file, np.uint8)
    img = cv2.imdecode(npimg,cv2.IMREAD_COLOR)
    ######### Do preprocessing here ################
    #img = ip.object_detection(img)
    ## any random stuff do here
    img[img > 150] = 0
    ################################################
    
    img = Image.fromarray(img.astype("uint8"))
    rawBytes = io.BytesIO()
    img.save(rawBytes, "JPEG")
    rawBytes.seek(0)
    img_base64 = base64.b64encode(rawBytes.read())
    return jsonify({'status':str(img_base64)})


# route http posts to this method
@app.route('/test', methods=['POST'])
def test():
    r = request
    # convert string of image data to uint8
    nparr = np.fromstring(r.data, np.uint8)
    # decode image
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    # do some fancy processing here....
    img.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)

    # build a response dict to send back to client
    response = {'message': 'image received. size={}x{}'.format(img.shape[1], img.shape[0])
                }
    # encode response using jsonpickle
    response_pickled = jsonpickle.encode(response)

    return Response(response=response_pickled, status=200, mimetype="application/json")

@app.after_request
def after_request(response):
    print("log: setting cors" , file = sys.stderr)
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
    response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE')
    return response

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e40cf70c85eb.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


log: setting cors
127.0.0.1 - - [04/Feb/2021 15:58:25] "GET / HTTP/1.1" 200 -
log: setting cors
127.0.0.1 - - [04/Feb/2021 15:58:26] "GET /static/index.js HTTP/1.1" 200 -
log: setting cors
127.0.0.1 - - [04/Feb/2021 15:58:26] "GET /static/favicon.ico HTTP/1.1" 200 -
